# Explore variation of Sentinel-1 shorelines across locations

## Load packages

In [1]:
%matplotlib inline

import os
os.environ['USE_PYGEOS'] = '0'
import datacube
import numpy as np
import xarray as xr
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.ops import nearest_points
from sklearn.metrics import RocCurveDisplay, roc_curve
from sklearn.metrics import accuracy_score,cohen_kappa_score,confusion_matrix,ConfusionMatrixDisplay,balanced_accuracy_score
from sklearn import tree

from deafrica_tools.datahandling import load_ard, mostcommon_crs
from deafrica_tools.bandindices import calculate_indices
from dea_tools.coastal import model_tides, tidal_tag, pixel_tides, tidal_stats
from dea_tools.spatial import subpixel_contours
from deafrica_tools.plotting import display_map, rgb, map_shapefile
from deafrica_tools.dask import create_local_dask_cluster
from coastlines.raster import tide_cutoffs,load_tidal_subset
from coastlines.vector import points_on_line, annual_movements, calculate_regressions

from scipy.ndimage.filters import uniform_filter
from scipy.ndimage.measurements import variance
from skimage.filters import threshold_minimum, threshold_otsu
from skimage.morphology import binary_closing, disk,diameter_closing
from datacube.utils.cog import write_cog

import itertools
import pickle

import warnings
warnings.filterwarnings("ignore")

from modules import lee_filter,filter_by_tide_height,load_s1_by_orbits,process_features_s1,create_coastal_mask,collect_training_samples,filter_obs_by_orbit
from datacube.utils.geometry import Geometry
from deafrica_tools.spatial import xr_rasterize
import random
from deafrica_tools.classification import predict_xr

In [2]:
# shorelines_ls_path="https://deafrica-services.s3.af-south-1.amazonaws.com/coastlines/v0.4.0/deafricacoastlines_v0.4.0.gpkg"
coastal_grid_path="https://deafrica-input-datasets.s3.af-south-1.amazonaws.com/deafrica-coastlines/32km_coastal_grid_deafrica.geojson"
# Set the range of dates for the analysis, time step and tide range
# time_range = ('2021')
time_range = ('2018', '2021')
time_step = '1Y'
tide_range = (0.25, 0.75)

# whether to apply orbit filtering
s1_orbit_filtering=True

# maximum number of samples per location
max_samples=10000

out_folder='results'
if not os.path.exists(out_folder):
    os.makedirs(out_folder)

In [3]:
## Load annual shorelines produced from Landsat
# %%time
# shorelines_annual = gpd.read_file(shorelines_ls_path,layer="shorelines_annual")
# shorelines_annual.head()
# shorelines_selected=shorelines_annual[(shorelines_annual.year>=int(time_range[0]))&(shorelines_annual.year<=int(time_range[1]))]
# shorelines_selected

## Load coastal grids

In [4]:
# Albers grid cells used to process the analysis
gridcell_gdf = gpd.read_file(coastal_grid_path).to_crs(epsg=6933).set_index('id')
gridcell_gdf.index=gridcell_gdf.index.astype(int)
gridcell_gdf.sort_values('id',inplace=True)
gridcell_gdf.head()

,left,top,right,bottom,geometry
id,,,,,
1,-2.479008e+06,2.157329e+06,-2.447008e+06,2.125329e+06,"MULTIPOLYGON (((-2474483.867 2125329.003, -247..."
2,-2.447008e+06,2.189329e+06,-2.415008e+06,2.157329e+06,"MULTIPOLYGON (((-2447008.208 2185622.468, -243..."
3,-2.447008e+06,2.157329e+06,-2.415008e+06,2.125329e+06,"MULTIPOLYGON (((-2447008.208 2157329.003, -241..."
4,-2.447008e+06,2.125329e+06,-2.415008e+06,2.093329e+06,"MULTIPOLYGON (((-2441834.147 2093329.003, -244..."
5,-2.415008e+06,2.189329e+06,-2.383008e+06,2.157329e+06,"MULTIPOLYGON (((-2402184.251 2189329.003, -239..."


## Set up a Dask cluster

In [5]:
client = create_local_dask_cluster(return_client=True)

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /user/whusggliuqx@gmail.com/proxy/8787/status,
Dashboard: /user/whusggliuqx@gmail.com/proxy/8787/status,Workers: 1
Total threads: 7,Total memory: 59.21 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:42491,Workers: 1
Dashboard: /user/whusggliuqx@gmail.com/proxy/8787/status,Total threads: 7
Started: Just now,Total memory: 59.21 GiB
Comm: tcp://127.0.0.1:42157,Total threads: 7
Dashboard: /user/whusggliuqx@gmail.com/proxy/39473/status,Memory: 59.21 GiB
Nanny: tcp://127.0.0.1:39503,


## Connect to the datacube

In [6]:
dc = datacube.Datacube(app="Sentinel-1 shoreline variation")

## Loop through coastal grids

In [7]:
gridcell_gdf_cp=gridcell_gdf.copy()
shoreline_buffer=300 # buffering distance (in metre) for coastal masking
for index in range(1,7):
# for index in gridcell_gdf.index:
    
    # check if current polygon has been processed
    outpath_roc=os.path.join(out_folder,'annual_shorelines_S1_supervised_thresh_polygon_{}.geojson'.format(str(index)))
    outpath_otsu=os.path.join(out_folder,'annual_shorelines_S1_otsu_thresh_polygon_{}.geojson'.format(str(index)))
    outpath_minim=os.path.join(out_folder,'annual_shorelines_S1_min_thresh_polygon_{}.geojson'.format(str(index)))
    outpath_s2=os.path.join(out_folder,'annual_shorelines_S2_polygon_{}.geojson'.format(str(index)))
    outpath_dt=os.path.join(out_folder,'annual_shorelines_S1_dt_polygon_{}.geojson'.format(str(index)))
    
    if ((os.path.exists(outpath_otsu))and(os.path.exists(outpath_minim))and(os.path.exists(outpath_s2))
    and(os.path.exists(outpath_roc))and(os.path.exists(outpath_dt))):
#     if (os.path.exists(outpath_otsu))and(os.path.exists(outpath_minim))and(os.path.exists(outpath_s2))and(os.path.exists(outpath_roc)):
#     if (os.path.exists(outpath_otsu))and(os.path.exists(outpath_minim))and(os.path.exists(outpath_s2)):
        print('Shoreline files exist, skipping this polygon...')
        continue
    else:
        # Load and process Sentinel-1 data
        print('querying polygon ',index)
        
        # Create query based on buffered analysis gridcell extent
        gdf = gridcell_gdf.loc[[index]]
        geopoly = Geometry(gdf.iloc[0].geometry, crs=gridcell_gdf.crs).buffer(100)
        query = {
            'geopolygon': geopoly,
            'time': time_range,
            'dask_chunks': {'time': 1},
            'measurements': ['vh','vv','mask'], # loading vv for experiement
            'resolution': (-20, 20),
            'output_crs':"EPSG:6933",
            'group_by':'solar_day',
        }
        
        # load s1 by orbit
        S1=load_s1_by_orbits(dc,query)
        
        # process unless no s1 data available
        if S1 is None:
            print('No Sentinel-1 data available, skipping polygon...')
            continue
        else:
            # per-pixel tide modelling and filtering
            print('Tide modelling and filtering for Sentinel-1...')
            S1_filtered=filter_by_tide_height(S1,tide_centre=0.0)

            # filter s1 observations by orbit if required
            if s1_orbit_filtering:
                S1_filtered=filter_obs_by_orbit(S1_filtered)

            # Scale to plot data in decibels
            S1_filtered['vh'] = 10 * np.log10(S1_filtered.vh)
            S1_filtered['vv'] = 10 * np.log10(S1_filtered.vv)

            # annual composites
            print('Generate temporal composites...')
            # median
            ds_summaries_s1 = (S1_filtered[['vh','vv']]
                               .resample(time=time_step)
                               .median('time').compute())
            ds_summaries_s1=ds_summaries_s1.rename({var:var+'_med' for var in list(ds_summaries_s1.data_vars)})

            # rename time as year
            ds_summaries_s1['time'] = ds_summaries_s1.time.dt.year
            ds_summaries_s1 = ds_summaries_s1.rename(time='year')
            
            # Load and process Sentinel-2 data
            # query S2
            print('\nQuerying Sentinel-2 data...')
            query.update({'measurements': ['green','swir_1']})
            S2 = load_ard(dc=dc,
                          products=['s2_l2a'],
                          resampling='bilinear',
                          min_gooddata=0.8,
                          align=(10, 10),
                          mask_filters=[("opening", 2), ("dilation", 5)],
                          **query)

            # Calculate S2 water index
            S2 = calculate_indices(S2, index='MNDWI', satellite_mission='s2')

            # per-pixel tide modelling and filtering
            print('Tide modelling and filtering for Sentinel-2...')
            S2_filtered=filter_by_tide_height(S2,tide_centre=0.0)

            # median of S2
            ds_summaries_s2 = S2_filtered.resample(time=time_step).median('time').compute()
            
            # rename time as year
            ds_summaries_s2['time'] = ds_summaries_s2.time.dt.year
            ds_summaries_s2 = ds_summaries_s2.rename(time='year')
            
            # extract annual shorelines from S2
            contour_gdf_s2=subpixel_contours(da=ds_summaries_s2['MNDWI'],
                                        z_values=0,
                                        dim='year',
                                        crs=ds_summaries_s1.geobox.crs,
                                        output_path=outpath_s2,
                                        min_vertices=15).set_index('year')

            # generate simplified coastal mask from rasterised annual shorelines
    #         clipped_shoreline=gpd.clip(shorelines_selected,gdf.iloc[[0]])
    #         clipped_shoreline['geometry']=clipped_shoreline['geometry'].buffer(shoreline_buffer)
            contour_gdf_s2['geometry']=contour_gdf_s2['geometry'].buffer(shoreline_buffer)
            coastal_mask = xr_rasterize(gdf=contour_gdf_s2,
                                    da=ds_summaries_s1.vh_med,
                                    transform=ds_summaries_s1.vh_med.geobox.transform,
                                    crs=ds_summaries_s1.vh_med.geobox.crs)
            # apply mask on S1
            s1_summaries_masked=ds_summaries_s1.where(coastal_mask)
        #     # generate simplified coastal mask from S2
        #     coastal_mask=create_coastal_mask(S2_filtered['MNDWI'],10)

        # --------------------------shoreline extration through thresholding------------------------------------#
            # Supervised - using Sentinel-2 training samples and ROC
            data,labels,S2_class_masked=collect_training_samples(ds_summaries_s2,ds_summaries_s1,coastal_mask,max_samples)
            if len(labels)==0:
                print('No coastal samples found, skipping this polygon...')
                continue
            else:
                # ROC using median vh
                s1_median_vh=data[:,0]
                fpr, tpr, thresholds  = roc_curve(labels, s1_median_vh, pos_label=0)
                optim_idx = np.argmax(tpr - fpr)
                threshold_roc = thresholds[optim_idx]
                print('The optimal threshold is ',threshold_roc)
                gridcell_gdf_cp.loc[index,'t_roc']=threshold_roc

                # supervised shoreline extraction using decsion tree and the two bands
                clf=tree.DecisionTreeClassifier(max_depth=2) # setting max_depth to prevent overfitting
                clf=clf.fit(data,labels)
                # apply to each year
                predicted=[]
                for i in range(ds_summaries_s1.dims['year']):
                    ds_s1_year=ds_summaries_s1.isel(year=i)
                    predicted_year = predict_xr(clf,ds_s1_year,proba=True,persist=False,clean=True).assign_coords(year=ds_s1_year.year.values).expand_dims('year')
                    predicted.append(predicted_year)
                predicted=xr.concat(predicted,dim='year')
                # covert the overall classification probabilities to water probabilities
                predicted['Probabilities']=predicted.Probabilities.where(predicted.Predictions==1,100-predicted.Probabilities)

                # Unsupervised - histogram based automatic threshold calculation
                # minimum thresholding
                threshold_m = threshold_minimum(s1_summaries_masked.vh_med.values[np.isfinite(s1_summaries_masked.vh_med.values)])
                # otsu thresholding
                threshold_o = threshold_otsu(s1_summaries_masked.vh_med.values[np.isfinite(s1_summaries_masked.vh_med.values)])
                print('threshold identified by minimum thresholding method: ',threshold_m)
                print('threshold identified by otsu thresholding method: ', threshold_o)
                # record thesholds
                gridcell_gdf_cp.loc[index,'t_min']=threshold_m
                gridcell_gdf_cp.loc[index,'t_otsu']=threshold_o

                # Extract and export shorelines from thresholding methods
                contour_gdf_threshold_roc=subpixel_contours(da=s1_summaries_masked.vh_med,
                                                z_values=threshold_roc,
                                                dim='year',
                                                crs=ds_summaries_s1.geobox.crs,
                                                output_path=outpath_roc,
                                                min_vertices=15).set_index('year')
                contour_gdf_threshold_o=subpixel_contours(da=s1_summaries_masked.vh_med,
                                                          z_values=threshold_o,
                                                          dim='year',
                                                          crs=ds_summaries_s1.geobox.crs,
                                                          output_path=outpath_otsu,
                                                          min_vertices=15).set_index('year')
                contour_gdf_threshold_min=subpixel_contours(da=s1_summaries_masked.vh_med,
                                                            z_values=threshold_m,
                                                            dim='year',
                                                            crs=ds_summaries_s1.geobox.crs,
                                                            output_path=outpath_minim,
                                                            min_vertices=15).set_index('year')
                contour_gdf = subpixel_contours(da=predicted.Probabilities,
                                                z_values=50,
                                                dim='year',
                                                crs=predicted.geobox.crs,
                                                output_path=outpath_dt,
                                                min_vertices=15).set_index('year')


querying polygon  1

Querying and loading Sentinel-1 ascending data...
Using pixel quality parameters for Sentinel 1
Finding datasets
    s1_rtc

Querying and loading Sentinel-1 descending data...
Using pixel quality parameters for Sentinel 1
Finding datasets
    s1_rtc
Applying pixel quality/cloud mask
Returning 39 time steps as a dask array
Tide modelling and filtering for Sentinel-1...
Creating reduced resolution 5000 x 5000 metre tide modelling array
Modelling tides using FES2014 tide model
Reprojecting tides into original array


100%|██████████| 39/39 [00:01<00:00, 31.89it/s]



Filtering Sentinel-1 product by orbit...


/usr/local/lib/python3.10/dist-packages/rasterio/warp.py:344: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  _reproject(


Generate temporal composites...

Querying Sentinel-2 data...
Using pixel quality parameters for Sentinel 2
Finding datasets
    s2_l2a
Counting good quality pixels for each time step
Filtering to 378 out of 566 time steps with at least 80.0% good quality pixels
Applying morphological filters to pq mask [('opening', 2), ('dilation', 5)]
Applying pixel quality/cloud mask
Returning 378 time steps as a dask array
Tide modelling and filtering for Sentinel-2...
Creating reduced resolution 5000 x 5000 metre tide modelling array
Modelling tides using FES2014 tide model
Reprojecting tides into original array


...

Operating in single z-value, multiple arrays mode
Writing contours to results/annual_shorelines_S2_polygon_5.geojson

subseting Sentinel-2 data to match Sentinel-1 availability...
Number of samples available:  10928
The optimal threshold is  -18.455849
predicting...
   probabilities...
predicting...
   probabilities...
threshold identified by minimum thresholding method:  -19.88971
threshold identified by otsu thresholding method:  -20.725807
Operating in single z-value, multiple arrays mode
Writing contours to results/annual_shorelines_S1_supervised_thresh_polygon_5.geojson
Operating in single z-value, multiple arrays mode
Writing contours to results/annual_shorelines_S1_otsu_thresh_polygon_5.geojson
Operating in single z-value, multiple arrays mode
Writing contours to results/annual_shorelines_S1_min_thresh_polygon_5.geojson
Operating in single z-value, multiple arrays mode
Writing contours to results/annual_shorelines_S1_dt_polygon_5.geojson
querying polygon  6

Querying and loading

100%|██████████| 39/39 [00:01<00:00, 31.38it/s]



Filtering Sentinel-1 product by orbit...
Generate temporal composites...

Querying Sentinel-2 data...
Using pixel quality parameters for Sentinel 2
Finding datasets
    s2_l2a
Counting good quality pixels for each time step
Filtering to 138 out of 291 time steps with at least 80.0% good quality pixels
Applying morphological filters to pq mask [('opening', 2), ('dilation', 5)]
Applying pixel quality/cloud mask
Returning 138 time steps as a dask array
Tide modelling and filtering for Sentinel-2...
Creating reduced resolution 5000 x 5000 metre tide modelling array
Modelling tides using FES2014 tide model
Reprojecting tides into original array


100%|██████████| 138/138 [00:03<00:00, 35.84it/s]


Operating in single z-value, multiple arrays mode
Writing contours to results/annual_shorelines_S2_polygon_6.geojson

subseting Sentinel-2 data to match Sentinel-1 availability...
Number of samples available:  134300
The optimal threshold is  -22.062664
predicting...
   probabilities...
predicting...
   probabilities...
threshold identified by minimum thresholding method:  -22.590046
threshold identified by otsu thresholding method:  -22.805973
Operating in single z-value, multiple arrays mode
Writing contours to results/annual_shorelines_S1_supervised_thresh_polygon_6.geojson
Operating in single z-value, multiple arrays mode
Writing contours to results/annual_shorelines_S1_otsu_thresh_polygon_6.geojson
Operating in single z-value, multiple arrays mode
Writing contours to results/annual_shorelines_S1_min_thresh_polygon_6.geojson
Operating in single z-value, multiple arrays mode
Writing contours to results/annual_shorelines_S1_dt_polygon_6.geojson


In [8]:
# export updated coastal gids
gridcell_gdf_cp.to_file(os.path.join(out_folder,'coastal_grids_with_thresholds.geojson'))

In [9]:
# Shut down Dask client now that we have processed the data we need
client.close()